In [1]:
import os

In [2]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    model_name: str
    parameters: dict
    target_column: str

In [26]:

from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """
        This function returns the ModelTrainerConfig object that contains the configuration for the model training process.

        Args:
            None

        Returns:
            ModelTrainerConfig: The ModelTrainerConfig object containing the configuration for the model training process

        """
        config = self.config.model_trainer
        params = self.params.GradientBoostedTreesLearner
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            model_name=config.model_name,
            parameters=params,
            target_column=schema.name,
        )
        return model_trainer_config

In [28]:
import pandas as pd
import os
from ThyroidProject import logger
import ydf

In [81]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        """
        trains the model using the training data and saves it to the artifacts directory
        """
        train_data = pd.read_csv(self.config.train_data_path) 
        templates = ydf.GradientBoostedTreesLearner.hyperparameter_templates() # Hyperparameter templates
        model = ydf.GradientBoostedTreesLearner(label=self.config.target_column,
                                                task=ydf.Task.CLASSIFICATION, **templates["better_defaultv1"], **self.config.parameters).train(train_data)
        model.save(os.path.join(self.config.root_dir, self.config.model_name))
        logger.info(f"Model saved to : {os.path.join(self.config.root_dir, self.config.model_name)}")
        info = model.describe()._html # Training Description of the model
        with open(os.path.join(self.config.root_dir, "model_description.html"), "w") as f:
            f.write(info) # storing the training description in a html file

In [82]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-02-03 19:40:18,156:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-02-03 19:40:18,160:INFO:common:yaml file: params.yaml loaded successfully]
[2024-02-03 19:40:18,166:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-02-03 19:40:18,171:INFO:common:created directory at :artifacts]
[2024-02-03 19:40:18,173:INFO:common:created directory at :artifacts/model_trainer]
[2024-02-03 19:40:18,207:INFO:generic_learner:Use 16 thread(s) for training]
Train model on 5756 examples
Model trained in 0:00:02.032013
[2024-02-03 19:40:20,319:INFO:1173580813:Model saved to : artifacts/model_trainer/model]
